In [60]:
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv('김치찌개데이터.csv')
df=df.drop(['시간'], axis=1)

df

,일시,카테고리,강수량(mm),일조합(hr),일조율(%),일사합(MJ/m2)
0,2021-01-01,치킨,0.0,6.5,67.7,9.54
1,2021-01-01,회,0.0,6.5,67.7,9.54
2,2021-01-01,찜탕,0.0,6.5,67.7,9.54
3,2021-01-01,분식,0.0,6.5,67.7,9.54
4,2021-01-01,돈까스/일식,0.0,6.5,67.7,9.54
...,...,...,...,...,...,...
1027449,2021-12-31,야식,0.0,9.0,93.8,11.70
1027450,2021-12-31,야식,0.0,9.0,93.8,11.70
1027451,2021-12-31,한식,0.0,9.0,93.8,11.70
1027452,2021-12-31,야식,0.0,9.0,93.8,11.70


In [61]:
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()
label.fit(df['일시'])
df['일시'] = label.transform(df['일시'])
label.fit(df['카테고리'])
df['카테고리'] = label.transform(df['카테고리'])

df

,일시,카테고리,강수량(mm),일조합(hr),일조율(%),일사합(MJ/m2)
0,0,10,0.0,6.5,67.7,9.54
1,0,15,0.0,6.5,67.7,9.54
2,0,9,0.0,6.5,67.7,9.54
3,0,3,0.0,6.5,67.7,9.54
4,0,1,0.0,6.5,67.7,9.54
...,...,...,...,...,...,...
1027449,357,6,0.0,9.0,93.8,11.70
1027450,357,6,0.0,9.0,93.8,11.70
1027451,357,14,0.0,9.0,93.8,11.70
1027452,357,6,0.0,9.0,93.8,11.70


In [62]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler() 
df['일조합(hr)'] = mms.fit_transform(pd.DataFrame(df['일조합(hr)']))
df['일조율(%)'] = mms.fit_transform(pd.DataFrame(df['일조율(%)']))
df['일사합(MJ/m2)'] = mms.fit_transform(pd.DataFrame(df['일사합(MJ/m2)']))
df


,일시,카테고리,강수량(mm),일조합(hr),일조율(%),일사합(MJ/m2)
0,0,10,0.0,0.492424,0.714135,0.313403
1,0,15,0.0,0.492424,0.714135,0.313403
2,0,9,0.0,0.492424,0.714135,0.313403
3,0,3,0.0,0.492424,0.714135,0.313403
4,0,1,0.0,0.492424,0.714135,0.313403
...,...,...,...,...,...,...
1027449,357,6,0.0,0.681818,0.989451,0.384363
1027450,357,6,0.0,0.681818,0.989451,0.384363
1027451,357,14,0.0,0.681818,0.989451,0.384363
1027452,357,6,0.0,0.681818,0.989451,0.384363


In [63]:
# 1. 데이터 분할
X = df.drop('카테고리', axis=1)
y = df['카테고리']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [66]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model = XGBClassifier(objective='reg:squarederror', 
                      n_estimators=1000, 
                      max_depth=10, 
                      learning_rate=0.5
                      )

model.fit(X_train, y_train)
pred = model.predict(X_test)
pred

array([ 1,  1,  1, ..., 10,  1,  1], dtype=int64)

In [67]:
accuracy_score(y_test, pred)

0.3269940513267721

In [42]:
X_train, y_train

(         일시  강수량(mm)   일조합(hr)    일조율(%)  일사합(MJ/m2)
 417014   69      0.0  0.598485  0.705696    0.545335
 913223  292      0.0  0.621212  0.800633    0.483574
 79501    14      0.2  0.000000  0.000000    0.069317
 18384     2      0.0  0.416667  0.598101    0.289093
 83227    14      0.2  0.000000  0.000000    0.069317
 ...     ...      ...       ...       ...         ...
 259178   42      0.0  0.696970  0.907173    0.483574
 365838   60      0.1  0.356061  0.434599    0.423784
 131932   22      0.0  0.401515  0.553797    0.319317
 671155  179     21.4  0.000000  0.000000    0.088371
 121958   20      9.2  0.000000  0.000000    0.090999
 
 [770590 rows x 5 columns],
 417014    14
 913223    12
 79501     10
 18384      7
 83227     14
           ..
 259178    11
 365838     7
 131932    12
 671155    12
 121958     1
 Name: 카테고리, Length: 770590, dtype: int32)

In [69]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

params = {
    'max_depth': [i for i in range(10)],===-
    'learning_rate': [i*0.01 for i in range(10)],
    'n_estimators': [i for i in range(100, 1000, 100)],
    'min_child_weight': [i for i in range(10)]
}
# 그리드 서치 수행
xgb = XGBClassifier(tree_method='gpu_hist')
grid_search = GridSearchCV(xgb, params, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 조합 출력
print(grid_search.best_params_)

KeyboardInterrupt: 

In [ ]:
# 최적 하이퍼파라미터 확인
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# 최적 하이퍼파라미터를 사용하여 모델 재학습
final_model = XGBClassifier(**best_params)
final_model.fit(X_train, y_train)

# 학습된 모델 저장
import joblib
joblib.dump(final_model, 'final_model.pkl')